Group 35 Project Proposal: Predicting Heart Disease

Introduction

The data below has been collected from 123 different patients and reports on specific aspects of their health with the purpose of determining the presence of heart disease in a patient. Similar data has been collected from hospitals in Budapest, Cleveland, and Switzerland, originally with 76 different variables. For the purposes of this project, the data used will only contain 14 variables, and is taken from the Hungarian Institute of Cardiology in Budapest. The question to be answered is: How present is heart disease in a patient, with categorical values ranging from 0 (no heart disease present) to 4?


Preliminary Data Analysis

In [8]:
# Load package
library(tidyverse)
library(digest)
library(repr)
library(tidymodels)
library(GGally)
library(ISLR)
options(repr.matrix.max.rows = 6)

Warning message:
“package ‘tidymodels’ was built under R version 4.0.2”
── Attaching packages ────────────────────────────────────── tidymodels 0.1.1 ──

✔ broom     0.7.0      ✔ recipes   0.1.13
✔ dials     0.0.9      ✔ rsample   0.0.7 
✔ infer     0.5.4      ✔ tune      0.1.1 
✔ modeldata 0.0.2      ✔ workflows 0.2.0 
✔ parsnip   0.1.3      ✔ yardstick 0.0.7 

Warning message:
“package ‘broom’ was built under R version 4.0.2”
Warning message:
“package ‘dials’ was built under R version 4.0.2”
Warning message:
“package ‘infer’ was built under R version 4.0.3”
Warning message:
“package ‘modeldata’ was built under R version 4.0.1”
Warning message:
“package ‘parsnip’ was built under R version 4.0.2”
Warning message:
“package ‘recipes’ was built under R version 4.0.1”
Warning message:
“package ‘tune’ was built under R version 4.0.2”
Warning message:
“package ‘workflows’ was built under R version 4.0.2”
Warning message:
“package ‘yardstick’ was built under R version 4.0.2”
── Conflicts ────

In [9]:
# Read in the heart disease data, rename the column name and mutate all columns to be numeric
url <- "https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/reprocessed.hungarian.data"
heart_disease <- read_delim(url, delim = " ", col_names = FALSE) %>%
                    rename(age = X1, sex = X2, chest_pain_type = X3, resting_BP = X4, cholesteral = X5, fasting_bs = X6, rest_ecg = X7,
                           max_hr = X8, ex_angina = X9, st_depression = X10, slope_st = X11, num_major_vessels = X12, thal = X13, diagnosis = X14)

Parsed with column specification:
cols(
  X1 = col_double(),
  X2 = col_double(),
  X3 = col_double(),
  X4 = col_double(),
  X5 = col_double(),
  X6 = col_double(),
  X7 = col_double(),
  X8 = col_double(),
  X9 = col_double(),
  X10 = col_double(),
  X11 = col_double(),
  X12 = col_double(),
  X13 = col_double(),
  X14 = col_double()
)



In [10]:
# Mutate the diagnosis, sex, chest_pain_type, fasting_bs, rest_ecg, ex_angina columns to be factors
heart_disease <- heart_disease %>%
                mutate(diagnosis = as.factor(diagnosis))

# Replace all "-9" as NA
heart_disease <- heart_disease %>%
                 mutate(across(where(is.numeric), ~na_if(., -9)))

# Select age, resting_BP, cholesteral, and amx_hr columns, remove any rows that contain NA
heart_disease_data <- heart_disease %>%
                      select(age, resting_BP, cholesteral, max_hr, diagnosis) %>%
                      na.omit() 

heart_disease_data

age,resting_BP,cholesteral,max_hr,diagnosis
<dbl>,<dbl>,<dbl>,<dbl>,<fct>
40,140,289,172,0
49,160,180,156,1
37,130,283,98,0
⋮,⋮,⋮,⋮,⋮
48,110,211,138,0
47,140,257,135,0
53,130,182,148,0


In [11]:
# Set the seed
set.seed(999) 

# Split the data into a training and test set

heart_disease_split <- initial_split(heart_disease_data, prop = 0.75, strata = diagnosis)
heart_disease_train <- training(heart_disease_split)
heart_disease_test <- testing(heart_disease_split)

heart_disease_train
heart_disease_test

age,resting_BP,cholesteral,max_hr,diagnosis
<dbl>,<dbl>,<dbl>,<dbl>,<fct>
40,140,289,172,0
49,160,180,156,1
39,120,339,170,0
⋮,⋮,⋮,⋮,⋮
51,110,190,120,0
36,120,166,180,0
48,110,211,138,0


age,resting_BP,cholesteral,max_hr,diagnosis
<dbl>,<dbl>,<dbl>,<dbl>,<fct>
37,130,283,98,0
48,138,214,108,3
54,120,273,150,0
⋮,⋮,⋮,⋮,⋮
55,110,344,160,0
47,140,257,135,0
53,130,182,148,0


In [15]:
# Create a ggpairs scatterplot of all the columns we are interested in including in our model
options(repr.plot.width = 15, repr.plot.height = 10)
heart_disease_eda <- ggpairs(heart_disease_train)

In [13]:
# Group by diagnosis and find average age, resting blood pressure ,cholesteral and maximum heart rate achieved, 
# the number of observations, and percentage in each group

heart_disease_train_table <- heart_disease_train %>%
                                select(age,resting_BP, max_hr, cholesteral, diagnosis)%>%
                                group_by(diagnosis)%>%
                                summarize(avg_age = round(mean(age), digits = 1),
                                          avg_resting_BP = round(mean(resting_BP)),
                                          avg_max_hr = round(mean(max_hr)),
                                          avg_cholesteral = round(mean(cholesteral)),
                                          num_obs = n(),
                                          percentage = round(n()/ nrow(heart_disease_train)*100,digits = 1))

heart_disease_train_table

`summarise()` ungrouping output (override with `.groups` argument)



diagnosis,avg_age,avg_resting_BP,avg_max_hr,avg_cholesteral,num_obs,percentage
<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<dbl>
0,47.1,130,146,244,129,63.5
1,48.2,128,136,246,28,13.8
2,48.7,139,126,275,17,8.4
3,52.6,140,118,289,17,8.4
4,50.3,139,121,298,12,5.9


In [ ]:
# Plot for age vs. resting blood pressure

options(repr.plot.width = 13, repr.plot.height = 8)

resting_BP_plot <- heart_disease_train %>%
                    ggplot(aes(x = age, y = resting_BP, color = diagnosis))+
                    geom_point(position = "jitter")+
                    labs(x = "Age", y = "Resting Blood Pressure(mm Hg on admission to the hospital)", colour = "Diagnosis Of Heart Disease")+
                    ggtitle("Age vs. Resting Blood Pressure")+
                    theme(text = element_text(size = 20))+
                    scale_y_continuous(labels = scales::comma)
resting_BP_plot

In [ ]:
# Plot for age vs. maximum heart rate achieved

options(repr.plot.width = 13, repr.plot.height = 8)

max_hr_plot <- heart_disease_train %>%
                    ggplot(aes(x = age, y = max_hr, color = diagnosis))+
                    geom_point(position = "jitter")+
                    labs(x = "Age", y = "Maximum Heart Rate Achieved", colour = "Diagnosis Of Heart Disease")+
                    ggtitle("Age vs. Maximum Heart Rate Achieved")+
                    theme(text = element_text(size = 20))+
                    scale_y_continuous(labels = scales::comma)
max_hr_plot

Methods

We will use KNN classification to predict the diagnosis. We will create a classifier, tune the classifier and visualize the results.

We will use the variables: 
age = (yrs)
sex = (1 = male, 0 = female)
chest_pain_type = (1-4)
resting_BP = resting blood pressure (mm Hg on admission to the hospital)
rest_ecg = resting electrocardiographic results
0: normal
1: having ST-T wave abnormality (T wave inversions and/or ST elevation or depression of > 0.05 mV)
2: showing probable or definite left ventricular hypertrophy by Estes' criteria
max_hr = maximum heart rate achieved
ex_angina = exercise induced angina (1 = yes; 0 = no)
st_depression = ST depression induced by exercise relative to rest
slope_st = the slope of the peak exercise ST segment (1: upsloping, 2: flat, 3: downsloping)
thal = Thalassemia (3: normal; 6: fixed defect; 7: reversible defect)
diagnosis = diagnosis of heart disease (Value 0: < 50% diameter narrowing, Value 1: > 50% diameter narrowing)

We will make a plot of predicted and true diagnosis values with a best fit line through the true values, of "a variable" vs "diagnosis"



Expected outcomes and Significance

First, we expect to find the best KNN classifier which has the highest accuracy that predicts diagnosis.
Second, we expect to find the best fitted line for the age and one (or several) specific aspect(s) of health. 

Assuming that a positive relationship exists between age and cholesterol level, for instance, there should be a best fitted line which has the least residues. And we can conclude that as people grow older, the possibility of obtaining a higher cholesterol becomes greater. If this conclusion holds true for age versus all other variables, we can say that people should pay more attention to heart disease as they grow older. 
This may lead to a few future questions. If there exists one or some variables that doesn’t have a linear relationship with sex, will the regression method we leaarnt still be working? Hence, what conclusion should we draw from our data? What’s more, since in our data, there’s also a column called “sex”, so what will happen if we take “sex” into consideration? Will our primary conclusion still hold true? If it does, how do the slopes of the best fitted line differ between  males and females? If it doesn’t, what will our further conclusion be like? 
